# A notebook to generate a CSV template with info from samples in location(s)

Give the output file name (with path)

In [2]:
filename="/Users/pierrespc/Desktop/2021-11-19_PETROUStoLumila.tsv"


## Preparing the note book

Please enter the one-line file where your token is saved in the following cell

In [5]:
tokenFile="/Users/pierrespc/Documents/PostDocPasteur/aDNA/Import_eLAB/API_FUNCTIONALITIES/credentials/tokenELAB"
import os
import json
import requests
import csv
import pandas
import numpy
from apiclient import discovery, errors
from httplib2 import Http
from oauth2client import client, file, tools
import os.path

token = format(open(tokenFile,"r").readline().strip())
url = "https://elab-dev.pasteur.fr/api/v1/"
headers1 = {'Authorization': token, 'Accept': 'application/json','Content-Type':'application/json'}
headers2 = {'Authorization': token, 'Accept': 'application/json'}



In [45]:
storageByID={}
r=requests.get(url+"/storageLayers",headers=headers1)
stoData=r.json().get("data")
for sto in stoData:
    storageByID[sto["storageLayerID"]]={"name":sto["name"],"parentID":sto["parentStorageLayerID"]}
    #print(sto["name"])
def getParentSto(ID,stoDict):
    if stoDict[ID]["parentID"]==0:
        return(stoDict[ID]["name"])
    else:
        return(getParentSto(stoDict[ID]["parentID"],stoDict)+", "+stoDict[ID]["name"])
    
    
storage={}
for stoID in storageByID.keys():
    name=getParentSto(stoID,storageByID)
    storage[name]=stoID
    



In [46]:
# Get storage
print(storageByID)

{774657: {'name': 'Nico office', 'parentID': 0}, 774658: {'name': 'In Copenhagen', 'parentID': 0}, 774659: {'name': 'Tom Gilbert Freezer', 'parentID': 0}, 774671: {'name': 'bag A1 + A2', 'parentID': 774659}, 774677: {'name': 'calculus extraction', 'parentID': 774671}, 774678: {'name': 'petrous back-up', 'parentID': 774671}, 774679: {'name': 'petrous extraction', 'parentID': 774671}, 774680: {'name': 'pulp back-up', 'parentID': 774671}, 774681: {'name': 'pulp extraction', 'parentID': 774671}, 774682: {'name': 'root back-up', 'parentID': 774671}, 774683: {'name': 'root extraction', 'parentID': 774671}, 774684: {'name': 'C group sensitive, blue box, back-up', 'parentID': 774671}, 775907: {'name': 'bag Mariano Del Papa calculus', 'parentID': 774659}, 775908: {'name': 'Mariano Del Papa calculus extraction', 'parentID': 775907}, 775909: {'name': 'already processed', 'parentID': 774671}, 774661: {'name': 'Freezer n9', 'parentID': 0}, 774674: {'name': 'Miren Drawer 2', 'parentID': 774661}, 774

In [73]:

iter=-1
prev="bliblablou"

print("get the storage where to look for samples:")
listID=[]
for sto in storageByID:
    if storageByID[sto]["parentID"] == 0:
        iter=iter+1
        listID.append(storageByID[sto]["name"])        
        print(format(iter)+": "+storageByID[sto]["name"])
print(listID)
StorageToReach={}
listChoice=input("what are your choice? (the number separated by <space>)").split()
print("now we will choice the location layers within the storage")
for i in listChoice:
    idSto=listID[int(i)]
    ###we will go from the lower layers to the upper...
    upperLevel=True
    for sto in storage:
        stoSplit=sto.split(", ")
        if stoSplit[0] == idSto and len(stoSplit)>1:
            prompt="?"
            while prompt not in ["y","n"]:
                prompt=input("select layer "+ sto+" ?( y/n)")
            if prompt == "y":
                upperLevel=False
                StorageToReach[sto]={"id":storage[sto]}
    if upperLevel:
        StorageToReach[idSto]={"id":storage[idSto]}
        
            
print(StorageToReach)
            
        





get the storage where to look for samples:
0: Nico office
1: In Copenhagen
2: Tom Gilbert Freezer
3: Freezer n9
4: Freezer 4
5: Unknown
6: Individual
7: Site
8: Sequencing
9: Hannes Freezer
10: In Tartu
11: With Lumila
['Nico office', 'In Copenhagen', 'Tom Gilbert Freezer', 'Freezer n9', 'Freezer 4', 'Unknown', 'Individual', 'Site', 'Sequencing', 'Hannes Freezer', 'In Tartu', 'With Lumila']
what are your choice? (the number separated by <space>)11
now we will choice the location layers within the storage
Please be aware that we will filter using  a OR function so upper storage level just overwrite any lower filter
{'With Lumila': {'id': 776238}}


In [85]:
import pandas as pd
out={"Sample":[],"Storage":[],"Type":[]}
for idSto in StorageToReach:
    r=requests.get(url+"/storageLayers/"+format(StorageToReach[idSto]["id"])+"/samples",headers=headers1)
    for sam in r.json().get("data"):
        out["Sample"].append(sam["name"])
        out["Storage"].append(idSto)
        out["Type"].append(sam["sampleType"]["name"])

out=pd.DataFrame(out)   
out

,Sample,Storage,Type
0,AR0154.1,With Lumila,Skeleton Element
1,AR0168.1,With Lumila,Skeleton Element
2,AR0174.1,With Lumila,Skeleton Element
3,AR0176.1,With Lumila,Skeleton Element
4,AR0244.2,With Lumila,Skeleton Element
...,...,...,...
186,AR0687.2,With Lumila,Skeleton Element
187,AR0692.1,With Lumila,Skeleton Element
188,AR0694.2,With Lumila,Skeleton Element
189,AR0695.1,With Lumila,Skeleton Element


In [86]:
out.to_csv(filename, sep='\t', na_rep='NA',mode='a')